In [1]:
import numpy as np
import time

In [2]:
test_input = """>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"""

In [77]:
import requests

URL = "https://adventofcode.com/{year}/day/{day}/input"
url = URL.format(year='2022', day=17)
cookies = {"session":"53616c7465645f5fd345d8c12076a357c8d31b2c0c4ad1c152eefbd5c1cf09e0c8d6d7f657fc558887d443d2d07fbc90728ada9b6a91afcecb9926ffb4981a93"}
req = requests.get(url, cookies=cookies)

if req.status_code != 200:
    print("Error. Got status:", req.status_code)
    print(req.text)
    exit(3)
else:
    my_input = req.text

In [29]:
class tetris :
    
    def __init__(self,this_input) :
        
        self.rocks = [
            self.rock0(),
            self.rock1(),
            self.rock2(),
            self.rock3(),
            self.rock4()]
        self.space = np.ones((7,1))*10
        self.height = -1
        self.moves = this_input.split("\n")[0]
        
    def rock0(self) :
        r0 = np.zeros((7,1))
        r0[2:6,0]=1
        return r0
    def rock1(self):
        r1 = np.zeros((7,3))
        r1[2:5,1]=1
        r1[3,0:3]=1
        return r1
    def rock2(self):
        r2 = np.zeros((7,3))
        r2[2:5,0]=1
        r2[4,0:3]=1
        return r2
    def rock3(self):
        r3 = np.zeros((7,4))
        r3[2,0:4]=1
        return r3
    def rock4(self):
        r4 = np.zeros((7,2))
        r4[2:4,0:2]=1
        return r4

    def print_rock(self) :
        for i in range(len(self.rocks)) : 
            print("rock",i)
            s = ""
            for il in range(self.rocks[i].shape[1]-1,-1,-1) :
                l = self.rocks[i][:,il]
                for el in l :
                    if el == 1:
                        s+="#"
                    else :
                        s+="."
                s+="\n"
            s+="\n"
            print(s)
            
    def __str__(self) :
        s = ""
        for il in range(self.space.shape[1]-1,-1,-1) :
            l = self.space[:,il]
            for el in l :
                if el == 10:
                    s+="#"
                elif el == 1 :
                    s+="@"
                else :
                    s+="."
            s+="\n"
        return s
    
    def new_rock(self,i):
        self.space = np.c_[self.space,self.rocks[i]]
        
    def move_down(self):
        lx,ly = np.where(self.space==1)
        for x, y in zip(lx,ly) :
            self.space[x,y] = 0 
        
        if len(lx) == 0 :
            return False
        
        for x, y in zip(lx,ly) :
            if self.space[x,y-1] == 10 :
                for xm, ym in zip(lx,ly) :
                    self.space[xm,ym] = 10
                return False
        for x, y in zip(lx,ly) :
            self.space[x,y-1] = 1
        return True
        
    def move_lat_right(self) :
        lx,ly = np.where(self.space==1)
        
        for x, y in zip(lx,ly) :
            if x+1 >= self.space.shape[0] :
                return False
            elif self.space[x+1,y] == 10 :
                return False        
        
        for x, y in zip(lx,ly) :
            self.space[x,y] = 0 
        for x, y in zip(lx,ly) :
            self.space[x+1,y] = 1
        return True
        
    def move_lat_left(self) :
        lx,ly = np.where(self.space==1)
        
        for x, y in zip(lx,ly) :
            if x-1 <0 :
                return False
            elif self.space[x-1,y] == 10 :
                return False        
        
        for x, y in zip(lx,ly) :
            self.space[x,y] = 0 
        for x, y in zip(lx,ly) :
            self.space[x-1,y] = 1
        return True
        
        
    def get_YMIN(self) :
        yslist = list(np.where(self.space[0,:]==10)[0])
        yslist.reverse()
        
        LENX,LENY = self.space.shape
        
        for ystart in yslist :
            path = [ystart]
            ymin = [ystart]
            for x in range(1,7) :
                npath = []
                nymin = []
                for y,ym in zip(path,ymin) :
                    yyp = y+1
                    yym = y-1
                    if self.space[x,y] == 10 :
                        npath.append(y)
                        nymin.append(ym)
                    while yyp < LENY and self.space[x,yyp] == 10 : 
                        npath.append(yyp)
                        nymin.append(ym)
                        yyp += 1
                    while yym >= 0 and self.space[x,yym] == 10 :
                        npath.append(yym)
                        nymin.append(min(ym,yym))
                        yym-=1
                path = npath
                ymin = nymin

            if len(path) > 0:
                self.height += max(ymin)
                return max(ymin)
         
    def get_YMIN_2(self) :
        yslist = list(np.where(self.space[0,:]==10)[0])
        yslist.reverse()
        
        LENX,LENY = self.space.shape
        
        for ystart in yslist :
            path = [ystart]
            ymin = [ystart]
            for x in range(1,7) :
                npath = []
                nymin = []
                for y,ym in zip(path,ymin) :
                    yyp = y+1
                    yym = y-1
                    if self.space[x,y] == 10 :
                        npath.append(y)
                        nymin.append(ym)
                    if yyp < LENY and self.space[x,yyp] == 10 : 
                        npath.append(yyp)
                        nymin.append(ym)
                    if yym >= 0 and self.space[x,yym] == 10 :
                        npath.append(yym)
                        nymin.append(min(ym,yym))
                        
                path = npath
                ymin = nymin

            if len(path) > 0:
                self.height += max(ymin)
                return max(ymin)
    
    
    def trim_space(self):
#         YMIN = self.get_YMIN()
#         YMIN = 0
        YMIN = self.get_YMIN_2()
        YMAX = np.where(t.space.sum(0)>0)[0][-1]+1
        self.space = self.space[:,YMIN:YMAX]
        
    def get_max_height(self):
        self.trim_space()
        return self.height + self.space.shape[1]
    
    def part2(self,Nrocks) :
        
        ir = 0
        ilm = 0
        st = time.perf_counter()
        for i in range(Nrocks) :
            
            if i % 10000 == 0 :
                print(i,self.get_max_height(),time.perf_counter()-st)
                st=time.perf_counter()
            
            # generate a rock
            self.new_rock(ir)
            ir += 1
            if ir == len(self.rocks) :
                ir = 0
            
            # 4 lateral moves first while it falls unimpeded
            for ii in range(4):
                if self.moves[ilm] == ">" :
                    self.move_lat_right()
                elif self.moves[ilm] == "<" :
                    self.move_lat_left()
                else :
                    print("pbm : invalid move input : ",self.moves[ilm])
                ilm += 1
                if ilm == len(self.moves) :
                    ilm = 0
            
            # move the rock
            while self.move_down() :
                if self.moves[ilm] == ">" :
                    self.move_lat_right()
                elif self.moves[ilm] == "<" :
                    self.move_lat_left()
                else :
                    print("pbm : invalid move input : ",self.moves[ilm])
                ilm += 1
                if ilm == len(self.moves) :
                    ilm = 0
                
            # trim the space
            self.trim_space()
            
        return self.get_max_height()

In [20]:
t = tetris(test_input)
t.part2(2)
# 4

0 0 0.00028410000959411263


4

In [21]:
t = tetris(test_input)
t.part2(10)
# 17

0 0 0.00016029999824240804


17

In [22]:
t = tetris(test_input)
t.part2(20)
# 36

0 0 9.220000356435776e-05


36

In [23]:
t = tetris(test_input)
t.part2(2022)
# 3068

0 0 0.00011650001397356391
1000 1520 2.8364785999874584
2000 3034 2.930152899993118


3068

In [26]:
t = tetris(test_input)
t.part2(2022)
# 3068

0 0 0.00015599996550008655
1000 1520 0.6511668000021018
2000 3034 0.7191009999951348


3068

In [27]:
t = tetris(test_input)
t.part2(10000)

0 0 6.049999501556158e-05
1000 1520 0.7192365999799222
2000 3034 0.7191680999821983
3000 4548 0.7062240000232123
4000 6061 0.7517736000008881
5000 7577 0.8016998999519274
6000 9088 0.7209074999555014
7000 10607 0.6582346999784932
8000 12120 0.7195994000067003
9000 13634 0.8242727000033483


15148

In [31]:
t = tetris(my_input)
t.part2(1000000000000)

0 0 0.00020730000687763095
10000 15704 9.777089600043837
20000 31393 9.121045199979562
30000 47082 9.471022500016261
40000 62752 6.561495299974922
50000 78451 9.62819820002187
60000 94143 9.622870600025635
70000 109814 9.82600470003672
80000 125504 9.643391399993561
90000 141196 9.737300600041635
100000 156865 9.712214400002267
110000 172558 9.880362300027627
120000 188256 9.762782600009814
130000 203919 9.762060299981385
140000 219621 9.556655000022147
150000 235303 9.26135179999983
160000 250975 9.474586400028784
170000 266679 9.754119399993215
180000 282357 9.668950500024948
190000 298025 9.526435300009325
200000 313732 9.759744499984663
210000 329419 9.143862100027036
220000 345087 9.987957399978768
230000 360783 10.32198279997101
240000 376472 10.175944400019944
250000 392133 10.002224700001534
260000 407842 9.814424499985762
270000 423522 10.066722999967169
280000 439200 10.290230699989479
290000 454887 10.550506800005678
300000 470581 9.29173809994245
310000 486243 9.65819440002

2470000 3874249 7.56898809998529
2480000 3889940 7.844965800002683
2490000 3905627 7.7296400999766774
2500000 3921295 7.5755329000530764
2510000 3937007 7.557825900032185
2520000 3952678 7.546982300002128
2530000 3968360 7.5128155000275
2540000 3984052 7.570539299980737
2550000 3999732 7.639658300031442
2560000 4015406 7.53251759998966
2570000 4031113 7.520121899957303
2580000 4046783 7.493053099955432
2590000 4062473 7.5016929999692366
2600000 4078158 7.558440199994948
2610000 4093839 7.632901299977675
2620000 4109521 7.531338500033598
2630000 4125222 7.503142400004435
2640000 4140893 7.587542499997653
2650000 4156586 7.403696699999273
2660000 4172275 7.51256000000285
2670000 4187953 7.510031500016339
2680000 4203633 7.520715899998322
2690000 4219329 7.4153074999921955
2700000 4235003 7.578188700019382
2710000 4250697 7.560473199991975
2720000 4266393 7.682886400027201
2730000 4282056 7.600308799999766
2740000 4297745 7.4570752999861725
2750000 4313437 7.4809278000029735
2760000 43291

4880000 7654372 7.620236499991734
4890000 7670042 7.6910910999868065
4900000 7685735 7.636321900004987
4910000 7701432 7.574812499980908
4920000 7717097 7.545186500006821
4930000 7732797 7.570748199999798
4940000 7748481 7.591834400023799
4950000 7764154 7.553831500001252
4960000 7779859 7.640215299965348
4970000 7795536 7.595895399979781
4980000 7811202 7.71291659999406
4990000 7826910 7.689085200021509
5000000 7842595 7.556649899983313
5010000 7858266 7.46809600002598
5020000 7873960 7.625311399984639
5030000 7889651 7.6703758999938145
5040000 7905313 7.744638200034387
5050000 7921019 7.92904020001879
5060000 7936701 7.654110800009221
5070000 7952375 7.666311400011182
5080000 7968066 7.877515500003938
5090000 7983756 7.5573725999565795
5100000 7999419 7.501331200008281
5110000 8015134 7.8129584999987856
5120000 8030816 7.627334799966775
5130000 8046491 7.466250800003763
5140000 8062183 7.900497000024188
5150000 8077872 7.769716300012078
5160000 8093537 7.673619399953168
5170000 81092

7270000 11403120 7.566281999985222
7280000 11418804 7.461915200052317
7290000 11434473 7.546289599966258
7300000 11450185 7.50858900003368
7310000 11465855 7.507840000034776
7320000 11481539 7.383829099999275
7330000 11497228 7.502717900031712
7340000 11512910 7.567890000005718
7350000 11528584 7.403764599992428
7360000 11544293 7.471139699977357
7370000 11559960 7.53076960000908
7380000 11575649 7.4740061000338756
7390000 11591338 7.55384279997088
7400000 11607016 7.497400899999775
7410000 11622698 7.518079599947669
7420000 11638398 7.497112900018692
7430000 11654069 7.498769300000276
7440000 11669765 7.440524199977517
7450000 11685454 7.644954000017606
7460000 11701131 7.554564100049902
7470000 11716809 7.531277499976568
7480000 11732510 7.429805899970233
7490000 11748179 7.539399400004186
7500000 11763873 7.415116099989973
7510000 11779571 7.512551599997096
7520000 11795236 7.5045688999816775
7530000 11810923 7.523665699991398
7540000 11826615 7.530643199977931
7550000 11842286 7.50

9620000 15089107 7.574124299979303
9630000 15104809 7.414295300026424
9640000 15120500 7.518755299970508
9650000 15136170 7.480797500000335
9660000 15151862 7.594146399991587
9670000 15167553 7.466318399994634
9680000 15183218 7.482479400001466
9690000 15198916 7.47242250002455
9700000 15214609 7.590017399983481
9710000 15230279 7.425946500035934
9720000 15245975 7.489938199985772
9730000 15261659 7.579844499996398
9740000 15277330 7.540962699975353
9750000 15293032 7.544234899978619
9760000 15308716 7.558885800011922
9770000 15324382 7.520139199972618
9780000 15340088 7.622175400028937
9790000 15355773 7.598187599971425
9800000 15371445 7.527914799982682
9810000 15387140 7.676263400004245
9820000 15402832 7.65394439996453
9830000 15418487 7.498026299988851
9840000 15434202 9.181893300032243
9850000 15449882 7.983531899983063
9860000 15465552 7.427447099995334
9870000 15481247 7.476372100005392
9880000 15496934 7.532000400009565
9890000 15512598 7.518982500012498
9900000 15528311 7.641

...

16470000 25833437 6.768642699986231
16480000 25849107 7.009497200022452
16490000 25864809 6.709754500014242
16500000 25880500 7.723702599992976
16510000 25896170 6.695800399989821
16520000 25911862 6.804392100020777
16530000 25927553 7.042668999987654
16540000 25943218 6.919208200008143
16550000 25958916 6.776515000034124
16560000 25974609 7.2817087000003085
16570000 25990279 6.743233899993356
16580000 26005975 6.86726989998715
16590000 26021659 7.0929110999568366
16600000 26037330 6.928663500002585
16610000 26053032 6.9724375999649055
16620000 26068716 6.77340240002377
16630000 26084382 7.119727999961469
16640000 26100088 6.769936400000006
16650000 26115773 6.887012600025628
16660000 26131445 6.922708199999761
16670000 26147140 7.333171300007962
16680000 26162832 6.980607100005727
16690000 26178487 6.8298346000374295
16700000 26194202 7.087715499976184
16710000 26209882 6.906879200018011
16720000 26225552 6.77460369997425
16730000 26241247 6.663731999986339
16740000 26256934 6.9843406

18750000 29409651 5.915959000005387
18760000 29425313 5.993347499985248
18770000 29441019 6.407561899977736
18780000 29456701 6.0183304999955
18790000 29472375 6.220502299955115
18800000 29488066 6.339660300000105
18810000 29503756 5.984725899994373
18820000 29519419 6.328322499990463
18830000 29535134 6.294224600016605
18840000 29550816 5.914678899978753
18850000 29566491 6.311493299959693
18860000 29582183 6.442601799964905
18870000 29597872 6.381556699983776
18880000 29613537 5.974336799990851
18890000 29629245 6.45438169996487
18900000 29644919 6.677335399959702
18910000 29660604 5.775945200002752
18920000 29676299 6.337547399976756
18930000 29691981 6.218181399977766
18940000 29707649 6.500364099978469
18950000 29723361 6.856061099970248
18960000 29739032 6.741898400010541
18970000 29754713 6.710602900013328
18980000 29770407 6.412404500006232
18990000 29786092 6.440092100005131
19000000 29801763 6.450214900018182
19010000 29817470 6.473033500020392
19020000 29833137 6.25624749995

21030000 32985855 6.368021100002807
21040000 33001539 6.267676900024526
21050000 33017228 6.470937999954913
21060000 33032910 6.508878499967977
21070000 33048584 6.638071900000796
21080000 33064293 6.320357800053898
21090000 33079960 5.73871380003402
21100000 33095649 6.46415120002348
21110000 33111338 6.0373351000016555
21120000 33127016 6.244313500006683
21130000 33142698 6.383582199981902
21140000 33158398 6.1513083999743685
21150000 33174069 6.19216769997729
21160000 33189765 6.340311399952043
21170000 33205454 6.448071499995422
21180000 33221131 6.404109099996276
21190000 33236809 6.245678699982818
21200000 33252510 6.250917099998333
21210000 33268179 6.3404185000108555
21220000 33283873 6.477683799981605
21230000 33299571 6.019388400018215
21240000 33315236 6.206015500007197
21250000 33330923 6.132614300004207
21260000 33346615 6.373823200003244
21270000 33362286 6.279790000000503
21280000 33377989 6.142441999982111
21290000 33393677 5.635070400021505
21300000 33409349 6.32657109

23310000 36562056 7.909742299991194
23320000 36577745 7.600430299993604
23330000 36593437 7.638095600006636
23340000 36609107 7.896888200019021
23350000 36624809 7.654559799993876
23360000 36640500 7.616336500039324
23370000 36656170 7.797226800001226
23380000 36671862 7.832670900039375
23390000 36687553 8.550844700017478
23400000 36703218 7.516172899981029
23410000 36718916 7.439588300010655
23420000 36734609 7.745785699982662
23430000 36750279 7.415455899958033
23440000 36765975 7.48398839996662
23450000 36781659 7.3946465000044554
23460000 36797330 8.008651499985717
23470000 36813032 7.937875199946575
23480000 36828716 8.128670100006275
23490000 36844382 7.772365299984813
23500000 36860088 7.759807000053115
23510000 36875773 7.865404699987266
23520000 36891445 7.907286799978465
23530000 36907140 7.658572600048501
23540000 36922832 7.443124999990687
23550000 36938487 6.929400200024247
23560000 36954202 7.37494660000084
23570000 36969882 6.400779799965676
23580000 36985552 8.603838499

KeyboardInterrupt: 

In [79]:
class tetris3 :
    
    def __init__(self,this_input) :
        
        spaceOx = [0,1,2,3,4,5,6]
        spaceOy = [0,0,0,0,0,0,0]
        self.spaceO = {0: [0,1,2,3,4,5,6]}
        self.moves = this_input.split("\n")[0]
        self.h = 0
        self.state = {}
        self.nrock = {}
 
    def rock(self,i) :
        h = self.h +1
        if i == 0 :
            rX = [2,3,4,5]
            rY = [h]*4
        elif i ==1 :
            rX = [2,3,3,3,4]
            rY = [h+1,h+1,h+2,h,h+1]
        elif i ==2 :
            rX = [2,3,4,4,4]
            rY = [h,h,h,h+1,h+2]
        elif i ==3 :
            rX = [2,2,2,2]
            rY = [h,h+1,h+2,h+3]
        elif i == 4 :
            rX = [2,3,2,3]
            rY = [h,h,h+1,h+1]
        return rX,rY
    
        
    def move_down(self,rX,rY):
        add_to_occupied = False
        for x,y in zip(rX,rY) :
            if y-1<=self.h and x in self.spaceO[y-1] :
                add_to_occupied = True
                break
        if add_to_occupied :
            for x,y in zip(rX,rY) :
                if y in self.spaceO.keys() :
                    self.spaceO[y].append(x)
                else :
                    self.spaceO[y] = [x]
            self.h = max(self.h,max(rY))
            return None, None
        return rX,[y-1 for y in rY]
        
    def move_lat_right(self,rX,rY) :
        if max(rX) == 6 :
            return rX,rY
        for x, y in zip(rX,rY) :
            if y <= self.h and x+1 in self.spaceO[y] :
                return rX,rY        
        return [x+1 for x in rX],rY
        
    def move_lat_left(self,rX,rY) :
        if min(rX) == 0 :
            return rX,rY
        for x, y in zip(rX,rY) :
            if y <= self.h and x-1 in self.spaceO[y] :
                return rX,rY        
        return [x-1 for x in rX],rY
        
    
    def trim_space(self):
        k = list(self.spaceO.keys())
        if max(k) - min(k) > 250 :
            for i in range(min(k),min(k)+50) :
                del self.spaceO[i]
                
    def print_some_lines(self,y_min,y_max) :
        s = ""
        for y in range(y_min,y_max+1):
            for x in range(7):
                if x in self.spaceO[y]:
                    s += "1"
                else :
                    s += "0"
            s += "|"
        return s
        
    
    def get_pattern(self,Nrocks) :
        
        ir = 0
        ilm = 0
        st = time.perf_counter()

        for i in range(Nrocks) :
            
            if i == 10000 :
                nrocks_sp = 10000
                h_sp = self.h
                pattern = self.print_some_lines(self.h-20,self.h)
            elif i > 10000 :
                if self.print_some_lines(self.h-20,self.h) == pattern :
                    print("found pattern")
                    nrocks_ep = i
                    h_ep = self.h
                    nrocks_inp = nrocks_ep-nrocks_sp
                    h_inp = h_ep-h_sp
                    ir_next = ir
                    ilm_next = ilm
                    return nrocks_inp, h_inp, ir_next, ilm_next, i
            
            if i % 1000 == 0 :
                print(i,self.h,time.perf_counter()-st)
                st=time.perf_counter()
            
            # generate a rock
            rX,rY = self.rock(ir)
            ir += 1
            if ir == 5 :
                ir = 0
            
            # 4 lateral moves first while it falls unimpeded
            for ii in range(4):
                if self.moves[ilm] == ">" :
                    rX,rY = self.move_lat_right(rX,rY)
                elif self.moves[ilm] == "<" :
                    rX,rY = self.move_lat_left(rX,rY)
                ilm += 1
                if ilm == len(self.moves) :
                    ilm = 0
            
            # move the rock
            while True :
                rX,rY = self.move_down(rX,rY)
                if rX is None :
                    break
                if self.moves[ilm] == ">" :
                    rX,rY=self.move_lat_right(rX,rY)
                elif self.moves[ilm] == "<" :
                    rX,rY=self.move_lat_left(rX,rY)
                ilm += 1
                if ilm == len(self.moves) :
                    ilm = 0
                
            # trim the space
            self.trim_space()
        
        return self.h
    
    def use_pattern(self, nrocks_inp, h_inp, ir_next, ilm_next, nrock_cur, Nrocks) :
        
        st=time.perf_counter()
        
        h_start = self.h
        n_pattern = int((Nrocks - nrock_cur) /nrocks_inp)
        h_added = h_inp * n_pattern
        nrock_cur += nrocks_inp * n_pattern
            
        
        ir = ir_next
        ilm = ilm_next
        
        for i in range(nrock_cur+1,Nrocks+1) :
            if i % 1000000 == 0 :
                print(i,self.h,time.perf_counter()-st)
                st=time.perf_counter()
            
            # generate a rock
            rX,rY = self.rock(ir)
            ir += 1
            if ir == 5 :
                ir = 0
            
            # 4 lateral moves first while it falls unimpeded
            for ii in range(4):
                if self.moves[ilm] == ">" :
                    rX,rY = self.move_lat_right(rX,rY)
                elif self.moves[ilm] == "<" :
                    rX,rY = self.move_lat_left(rX,rY)
                ilm += 1
                if ilm == len(self.moves) :
                    ilm = 0
            
            # move the rock
            while True :
                rX,rY = self.move_down(rX,rY)
                if rX is None :
                    break
                if self.moves[ilm] == ">" :
                    rX,rY=self.move_lat_right(rX,rY)
                elif self.moves[ilm] == "<" :
                    rX,rY=self.move_lat_left(rX,rY)
                ilm += 1
                if ilm == len(self.moves) :
                    ilm = 0
                
            # trim the space
            self.trim_space()
            
        return self.h + h_added
        
    def part2(self,Nrocks):
        nrocks_inp, h_inp, ir_next, ilm_next, nrock_cur = self.get_pattern(Nrocks)
        return self.use_pattern(nrocks_inp, h_inp, ir_next, ilm_next, nrock_cur,Nrocks)
        

In [44]:
t = tetris3(test_input)
t.part2(2022)
# 3068

0 0 2.300017513334751e-06


3068

In [45]:
t = tetris3(test_input)
t.part2(10000)
# 15148

0 0 4.399975296109915e-06


15148

In [80]:
t = tetris3(test_input)
t.part2(100000)
# 151434

0 0 4.999979864805937e-06
1000 1520 0.043536100012715906
2000 3034 0.034670300025027245
3000 4548 0.029318499960936606
4000 6061 0.019056699995417148
5000 7577 0.03262359998188913
6000 9088 0.02710740000475198
7000 10607 0.03133590001380071
8000 12120 0.03478059999179095
9000 13634 0.033767699962481856
10000 15148 0.04938189999666065
found pattern


151434

In [81]:
t = tetris3(test_input)
t.part2(1000000)
# 1514288

0 0 4.999979864805937e-06
1000 1520 0.029281600029207766
2000 3034 0.04000940005062148
3000 4548 0.029638600011821836
4000 6061 0.059146099956706166
5000 7577 0.04402400000253692
6000 9088 0.025731700006872416
7000 10607 0.044626900053117424
8000 12120 0.027435199997853488
9000 13634 0.018541800032835454
10000 15148 0.029345799994189292
found pattern
1000000 15236 0.0008891000179573894


1514288

In [82]:
t = tetris3(test_input)
t.part2(1000000000000)
# should be : 1514285714288
#             1514285714288

0 0 6.89999433234334e-06
1000 1520 0.03153420001035556
2000 3034 0.023771399981342256
3000 4548 0.04040150000946596
4000 6061 0.03880939999362454
5000 7577 0.038720899960026145
6000 9088 0.04483079997589812
7000 10607 0.05660299997543916
8000 12120 0.04761480004526675
9000 13634 0.03544589999364689
10000 15148 0.05311219999566674
found pattern
1000000000000 15236 0.0010091000003740191


1514285714288

In [83]:
t = tetris3(my_input)
t.part2(1000000000000)
# too high
# 1612499999584

0 0 6.699992809444666e-06
1000 1600 0.04176980000920594
2000 3165 0.03684350004186854
3000 4714 0.021894000004976988
4000 6286 0.025226099998690188
5000 7872 0.030192500038538128
6000 9437 0.026103800046257675
7000 11006 0.022941700008232147
8000 12570 0.02281690004747361
9000 14139 0.022875400027260184
10000 15704 0.01899579999735579
11000 17278 0.06705710000824183
found pattern
1000000000000 18889 0.0061505999765358865


1568513119571

In [ ]:
1495000000 2344927140 31.25120340002468
10000000 15685158 43.98456430004444
11000000 17253677 41.55330050003249
12000000 18822183 40.09833190002246
13000000 20390697 42.81940560002113
14000000 21959200 43.949145600025076